In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown --id 1INnRuskYpNbUdS3dBsmr7EfmTEH_k1VK

Downloading...
From: https://drive.google.com/uc?id=1INnRuskYpNbUdS3dBsmr7EfmTEH_k1VK
To: /content/cropped_img.zip
100% 826M/826M [00:07<00:00, 110MB/s] 


In [ ]:
!unzip -q cropped_img.zip

warning [cropped_img.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [cropped_img.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #204203).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [ ]:
!gdown --id 12eX_LpjZBsZ5ndnpSlNwOub_3GcbRPwX

Downloading...
From: https://drive.google.com/uc?id=12eX_LpjZBsZ5ndnpSlNwOub_3GcbRPwX
To: /content/final_test.zip
100% 163M/163M [00:05<00:00, 28.0MB/s]


In [ ]:
!unzip -q final_test.zip

In [ ]:
!mv final_test cropped_img/final_test

In [ ]:
!mv final_test.csv cropped_img

In [ ]:
!pip install timm

     |████████████████████████████████| 431 kB 5.2 MB/s 


In [ ]:
import torch
from torch.utils.data import DataLoader

import timm
from timm.models.efficientnet import default_cfgs 

import os
import shutil
import sys
import time
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tqdm import tqdm

device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")

font_label = ['TH_Charm_of_AU', 'FC_Knomphing_Regular', 'fonttintin',
       'TH_Mali_Grade6', 'Angsana_New', 'Cordia_New', 'TH_Chakra_Petch',
       'TH_Sarabun', 'DM_Shining_Star_Regular', 'Kunlasatri']

font_label2 = ['TH Charm of AU.ttf',
     'FC Knomphing Regular.ttf',
     'fonttintin.ttf',
     'TH Mali Grade6.ttf',
     'ANGSA.ttf',
     'CORDIA.ttf',
     'TH Chakra Petch.ttf',
     'THSarabun.ttf',
     'DM Shining Star Regular.ttf',
     'Kunlasatri.ttf']

size_label = ['12px','14px','16px','18px','20px','22px','24px']

style_label = ['normal', 'italic']

weight_label = ["normal","bold"]


In [ ]:
df = pd.read_csv('./cropped_img/test.csv')
df_prediction = pd.read_csv('./sample_submission.csv')
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Id,imgId,OrgImgH,OrgImgW,CroppedImgH,CroppedImgW
0,b68ab614,1001,1684,1191,29,287
1,2d3a3417,1001,1684,1191,21,293
2,b9b73e2d,1001,1684,1191,25,29
3,1d8598b0,1001,1684,1191,71,176
4,6626584d,1001,1684,1191,11,59
...,...,...,...,...,...,...
48246,69f904fa,2000,1684,1191,12,217
48247,c44f7fec,2000,1684,1191,17,7
48248,3bec375c,2000,1684,1191,15,50
48249,ea90a783,2000,1684,1191,11,41


In [ ]:
df_final = pd.read_csv('./cropped_img/final_test.csv')
df_final

,Id,imgId,OrgImgH,OrgImgW,CroppedImgH,CroppedImgW
0,d0084406,2001,1516,1191,37,295
1,8812ecbf,2001,1516,1191,27,209
2,04ef28c7,2001,1516,1191,13,38
3,05aadb32,2001,1516,1191,40,213
4,a26e1f30,2001,1516,1191,12,15
...,...,...,...,...,...,...
47741,21c94748,3000,1389,1191,9,4
47742,786f10d3,3000,1389,1191,13,77
47743,da3d3427,3000,1389,1191,20,103
47744,990104af,3000,1389,1191,13,62


In [ ]:
# Dataset Loader
img_size = 208

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self,df,img_size = img_size,img_dir="./cropped_img/test"):
        self.img_path = list(df['Id'])
        self.img_size = img_size
        self.img_dir = img_dir
    def __len__(self):
        return len(self.img_path)
    def __getitem__(self, idx):
        img = self.load_preprocess_img("%s/%s.jpg"%(self.img_dir,self.img_path[idx]))
        return {'image': torch.FloatTensor(img),'id':self.img_path[idx]}
    
    def load_preprocess_img(self,img_path):
        img = cv2.imread(img_path)/255
        return cv2.resize(img, (self.img_size,self.img_size), interpolation = cv2.INTER_AREA)


test_dataset = ImageDataset(df)
test_final_dataset = ImageDataset(df_final, img_dir = "./cropped_img/final_test")

# dataloaders
batch_size = 768
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test_final_loader = DataLoader(test_final_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class ClassifierModel(torch.nn.Module):
    def __init__(self,n_dim1 = 256,num_class=10,dropout_rate=0.3):
        super(ClassifierModel, self).__init__() 
        self.effnv2_b3_model = timm.create_model(
                    'tf_efficientnetv2_b2',
                    pretrained=True)
        self.droupout1 = torch.nn.Dropout(p=dropout_rate)
        self.droupout2 = torch.nn.Dropout(p=dropout_rate)

        self.fc1 = torch.nn.Linear(1000, n_dim1)
        self.fc2 = torch.nn.Linear(n_dim1, num_class)

    def forward(self, x):
        x = torch.swapaxes(x, 1, -1)
        x = self.effnv2_b3_model(x)
        x = self.droupout1(x)
        x = self._mlp(x)

        return x
    
    def _mlp(self,x):
        x = self.fc1(x)
        x = torch.nn.functional.leaky_relu(x)
        x = self.droupout2(x)
        x = self.fc2(x)
        return x

In [ ]:
class MultiClassifierModel(torch.nn.Module):
  def __init__(self,model_path=[],num_class = 10):
    super(MultiClassifierModel, self).__init__() 
    self.num_class = num_class
    self.models = []
    for i in model_path:
      model = ClassifierModel(num_class=num_class)
      model.load_state_dict(torch.load(i)['state_dict'])
      model.eval().cuda()
      self.models += [model]
  def forward(self,x):
      outputs = []
      for model in self.models:
        logits = model(x)
        output = torch.nn.functional.softmax(logits,dim=1)
        outputs+=[output]
      return sum(outputs)/len(self.models)

font_classifier_model = MultiClassifierModel(num_class=10, 
                                             model_path=['/content/drive/Shareddrives/SUPERAIAppMan/weights/fontFamily/best_checkpoint_fold_0.pth.tar',
                                                         '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontFamily/best_checkpoint_fold_1.pth.tar', 
                                                         '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontFamily/best_checkpoint_fold_2.pth.tar', 
                                                         '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontFamily/best_checkpoint_fold_3.pth.tar', 
                                                         '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontFamily/best_checkpoint_fold_4.pth.tar', 
                                                         '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontFamily/best_checkpoint_org.pth.tar'])
font_classifier_model.eval().cuda()

weight_classifier_model = MultiClassifierModel(num_class=2, 
                                               model_path=['/content/drive/Shareddrives/SUPERAIAppMan/weights/fontWeight/checkpoint_model_fold_0/best_checkpoint.pth.tar',
                                                           '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontWeight/checkpoint_model_fold_1/best_checkpoint.pth.tar', 
                                                           '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontWeight/checkpoint_model_fold_2/best_checkpoint.pth.tar',
                                                           '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontWeight/checkpoint_model_fold_3/best_checkpoint.pth.tar',
                                                           '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontWeight/checkpoint_model_fold_4/best_checkpoint.pth.tar',
                                                           '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontWeight/checkpoint_eff_weightclassifier.pth.tar'])
weight_classifier_model.eval().cuda()

font_size_classifier_model = MultiClassifierModel(num_class=7, 
                                                  model_path=['/content/drive/Shareddrives/SUPERAIAppMan/weights/fontSize_fine-tune/checkpoint_model_fold_0/best_checkpoint.pth.tar', 
                                                              '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontSize_fine-tune/checkpoint_model_fold_1/best_checkpoint.pth.tar', 
                                                              '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontSize_fine-tune/checkpoint_model_fold_2/best_checkpoint.pth.tar',
                                                              '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontSize/checkpoint_model_fold_3/best_checkpoint.pth.tar', 
                                                              '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontSize_fine-tune/checkpoint_model_fold_4/best_checkpoint.pth.tar', 
                                                              '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontSize/checkpoint_fontSize.tar'])
font_size_classifier_model.eval().cuda()

style_classifier_model = MultiClassifierModel(num_class=2, 
                                              model_path=['/content/drive/Shareddrives/SUPERAIAppMan/weights/fontStyle/checkpoint_model_fold_0/best_checkpoint.pth.tar',
                                                            '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontStyle/checkpoint_model_fold_1/best_checkpoint.pth.tar', 
                                                            '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontStyle/checkpoint_model_fold_2/best_checkpoint.pth.tar', 
                                                            '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontStyle/checkpoint_model_fold_3/best_checkpoint.pth.tar', 
                                                            '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontStyle/checkpoint_model_fold_4/best_checkpoint.pth.tar', 
                                                            '/content/drive/Shareddrives/SUPERAIAppMan/weights/fontStyle/best_checkpoint.pth.tar'])
style_classifier_model.eval().cuda()


print("Load Model Success")

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_b2-847de54e.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnetv2_b2-847de54e.pth


Load Model Success


In [ ]:
def stream(message) :
    try:
        sys.stdout.write("\r{%s}" % message)
    except:
        message = ''.join(i for i in message if ord(i)<128)
        sys.stdout.write("\r{%s}" % message)
prediction_result = dict()

for i, batch in enumerate(test_loader):
    stream(f"{i+1}/{len(test_loader)}")
    img = batch['image'].to(device)
    # Prediction
    with torch.no_grad():
        font_logit = font_classifier_model(img)
        weight_logit = weight_classifier_model(img)
        font_size_logit = font_size_classifier_model(img)
        style_logit = style_classifier_model(img)
        
    font_predicted = list(font_logit.argmax(axis=1).cpu().numpy())
    weight_predicted = list(weight_logit.argmax(axis=1).cpu().numpy())
    font_size_predicted = list(font_size_logit.argmax(axis=1).cpu().numpy())
    style_predicted = list(style_logit.argmax(axis=1).cpu().numpy())
    
    prediction_features = np.array([batch['id'],font_predicted,weight_predicted,font_size_predicted,style_predicted]).T.tolist()
    for feat in prediction_features:
        idx,font_pred,weight_pred,font_size_pred,style_pred = feat
        prediction_result[idx+"_name"] = font_label[int(font_pred)]    
        prediction_result[idx+"_file"] = font_label2[int(font_pred)]
        prediction_result[idx+"_size"] = size_label[int(font_size_pred)]
        prediction_result[idx+"_style-weight"] =  style_label[int(style_pred)]+ "-" + weight_label[int(weight_pred)] 

{63/63}

In [ ]:
for i, batch in enumerate(test_final_loader):
    stream(f"{i+1}/{len(test_final_loader)}")
    img = batch['image'].to(device)
    # Prediction
    with torch.no_grad():
        font_logit = font_classifier_model(img)
        weight_logit = weight_classifier_model(img)
        font_size_logit = font_size_classifier_model(img)
        style_logit = style_classifier_model(img)
        
    font_predicted = list(font_logit.argmax(axis=1).cpu().numpy())
    weight_predicted = list(weight_logit.argmax(axis=1).cpu().numpy())
    font_size_predicted = list(font_size_logit.argmax(axis=1).cpu().numpy())
    style_predicted = list(style_logit.argmax(axis=1).cpu().numpy())
    
    prediction_features = np.array([batch['id'],font_predicted,weight_predicted,font_size_predicted,style_predicted]).T.tolist()
    for feat in prediction_features:
        idx,font_pred,weight_pred,font_size_pred,style_pred = feat
        prediction_result[idx+"_name"] = font_label[int(font_pred)]    
        prediction_result[idx+"_file"] = font_label2[int(font_pred)]
        prediction_result[idx+"_size"] = size_label[int(font_size_pred)]
        prediction_result[idx+"_style-weight"] =  style_label[int(style_pred)]+ "-" + weight_label[int(weight_pred)] 

{63/63}

In [ ]:
df_result = pd.Series(prediction_result).to_frame()
df_result = df_result.reset_index(drop=False)
df_result.columns = ["Id","Predicted"]
df_final_pred = pd.merge(df_prediction[['Id']], df_result, how='outer',on='Id')
df_final_pred.to_csv("/content/drive/Shareddrives/SUPERAIAppMan/04-TestPrediction-ensemble.csv",index=None)
df_final_pred

,Id,Predicted
0,b68ab614_name,TH_Chakra_Petch
1,b68ab614_file,TH Chakra Petch.ttf
2,b68ab614_size,24px
3,b68ab614_style-weight,normal-bold
4,2d3a3417_name,TH_Chakra_Petch
...,...,...
383983,990104af_style-weight,italic-normal
383984,41af819d_name,TH_Mali_Grade6
383985,41af819d_file,TH Mali Grade6.ttf
383986,41af819d_size,14px
